# Case 1: Evaluating Local Performances using Z-Score

## Import libraries and load Data

In [1]:
# This code begins by importing several libraries that are commonly used in data analysis and statistical modelling tasks
# The 'pandas' library is used for working with structured data, such as data stored in spreadsheets or CSV files
# The 'numpy' library is used for numerical computing, such as performing mathematical operations on arrays of numbers
# The 'scipy' library is used for scientific computing tasks, including statistical analysis and optimization
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# This code reads data from a CSV file named "study_case.csv" into a pandas dataframe called "data"
# To accomplish this, we use the "read_csv" function provided by the pandas library
data=pd.read_csv("study_case.csv")

# Next, we use string interpolation to print the shape of the data to the console
# We use the "shape" attribute of the dataframe, which returns a tuple containing the number of rows and columns in the dataframe
# The f-string allows us to include the value of "data.shape" within the text of the printed statement
print(f"Shape of data : {data.shape}")

# Finally, we print the first few rows of the data using the "head()" function of the dataframe
# This allows us to quickly examine the contents of the dataframe and ensure that the data has been loaded correctly
data.head()

Shape of data : (44196, 4)


,country,date,feature,value
0,AE,2022-01-01,ActiveSubscription,175
1,AE,2022-01-01,NewSubscription,1
2,AE,2022-01-02,ActiveSubscription,176
3,AE,2022-01-02,NewSubscription,1
4,AE,2022-01-03,ActiveSubscription,178


In [3]:
def missingpercentage(df):
    '''
    A function to calculate the percentage of missing values in each column of a pandas dataframe.

    Parameters:
    df (pandas dataframe): The dataframe for which the percentage of missing values needs to be calculated.

    Returns:
    missing_data (pandas dataframe): A dataframe with two columns: "Total" which shows the total number of missing values 
    in each column, and "Percent" which shows the percentage of missing values in each column.
    '''
    # calculate the total number of missing values in each column
    total = df.isnull().sum().sort_values(ascending=False)

    # calculate the percentage of missing values in each column
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False).round(2)

    # concatenate the total and percentage values into a new dataframe
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

    return missing_data

In [4]:
missingpercentage(data)

,Total,Percent
country,323,0.73
date,0,0.00
feature,0,0.00
value,0,0.00


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44196 entries, 0 to 44195
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  43873 non-null  object
 1   date     44196 non-null  object
 2   feature  44196 non-null  object
 3   value    44196 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [6]:
# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'])

# Drop rows with missing values
data.dropna(inplace=True)

# Set the 'date' column as the index
data.set_index("date", drop=True, inplace=True)

# Print the shape of the dataframe
print(f"Shape of data: {data.shape}")

# Print the first few rows of the dataframe
data.head()

Shape of data: (43873, 3)


,country,feature,value
date,,,
2022-01-01,AE,ActiveSubscription,175
2022-01-01,AE,NewSubscription,1
2022-01-02,AE,ActiveSubscription,176
2022-01-02,AE,NewSubscription,1
2022-01-03,AE,ActiveSubscription,178


## ActiveSubs Table

In [7]:
# Filter the 'data' dataframe to include only rows with 'feature' equal to 'ActiveSubscription'
active = data[data['feature'] == 'ActiveSubscription']

# Print the shape of the 'active' dataframe
print(f"Shape of active: {active.shape}")

# Print the first few rows of the 'active' dataframe
active.head()

Shape of active: (35400, 3)


,country,feature,value
date,,,
2022-01-01,AE,ActiveSubscription,175
2022-01-02,AE,ActiveSubscription,176
2022-01-03,AE,ActiveSubscription,178
2022-01-04,AE,ActiveSubscription,176
2022-01-05,AE,ActiveSubscription,175


In [8]:
# Resample the 'active' dataframe by grouping by 'country' and resampling to weekly frequency starting on Monday
# Use 'last' to fill missing values with the last valid observation
active_resampled = active.groupby('country').resample('W-MON', label='left', closed='left').last()

# Print the shape of the 'active_resampled' dataframe
print(f"Shape of active_resampled: {active_resampled.shape}")

# Print the first few rows of the 'active_resampled' dataframe
active_resampled.head()


Shape of active_resampled: (5703, 3)


country             feature  value
country date                                         
AE      2021-12-27      AE  ActiveSubscription  176.0
        2022-01-03      AE  ActiveSubscription  175.0
        2022-01-10      AE  ActiveSubscription  173.0
        2022-01-17      AE  ActiveSubscription  166.0
        2022-01-24      AE  ActiveSubscription  164.0

In [9]:
# Drop the "country" column since it's no longer needed
active_resampled.drop("country", axis=1, inplace=True)

# Reset the index to create a "Week" column
active_resampled.reset_index(inplace=True)

# Drop the "feature" column since it's no longer needed
active_resampled.drop("feature", axis=1, inplace=True)

# Rename the columns to make them more descriptive
active_resampled.rename(columns={"date": "Week", "value": "ActiveSubs"}, inplace=True)

# Print the shape of the resampled and transformed data
print(f"Shape of active_resampled: {active_resampled.shape}")

# Print the head of the resampled and transformed data
active_resampled.head()

Shape of active_resampled: (5703, 3)


,country,Week,ActiveSubs
0,AE,2021-12-27,176.0
1,AE,2022-01-03,175.0
2,AE,2022-01-10,173.0
3,AE,2022-01-17,166.0
4,AE,2022-01-24,164.0


## NewSubs Table


In [10]:
# Extract rows with feature = "NewSubscription"
new_subs = data[data["feature"] == "NewSubscription"]
# Print shape of new_subs DataFrame
print(f"Shape of new_subs: {new_subs.shape}")
# Display the first few rows of new_subs DataFrame
new_subs.head()

Shape of new_subs: (8473, 3)


,country,feature,value
date,,,
2022-01-01,AE,NewSubscription,1
2022-01-02,AE,NewSubscription,1
2022-01-04,AE,NewSubscription,2
2022-01-06,AE,NewSubscription,3
2022-01-09,AE,NewSubscription,1


In [11]:
# Group new_subs DataFrame by country and resample to weekly frequency starting on Monday
new_subs_resampled = new_subs.groupby("country").resample("W-MON", label="left", closed="left").sum()

# Print shape of new_subs_resampled DataFrame
print(f"Shape of new_subs_resampled: {new_subs_resampled.shape}")

# Print first five rows of new_subs_resampled DataFrame
new_subs_resampled.head()

C:\Users\bal_ozgur\AppData\Local\Temp\ipykernel_4840\3517973491.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  new_subs_resampled = new_subs.groupby("country").resample("W-MON", label="left", closed="left").sum()


Shape of new_subs_resampled: (3966, 1)


value
country date             
AE      2021-12-27      2
        2022-01-03      6
        2022-01-10     12
        2022-01-17      5
        2022-01-24      6

In [12]:
# Reset the index of new_subs_resampled DataFrame
new_subs_resampled.reset_index(inplace=True)

# Rename columns for better readability
new_subs_resampled.rename(columns={"date": "Week", "value": "NewSubs"}, inplace=True)

# Print shape of the updated new_subs_resampled DataFrame
print(f"Shape of new_subs_resampled : {new_subs_resampled.shape}")

# Print the first five rows of the updated new_subs_resampled DataFrame
new_subs_resampled.head()


Shape of new_subs_resampled : (3966, 3)


,country,Week,NewSubs
0,AE,2021-12-27,2
1,AE,2022-01-03,6
2,AE,2022-01-10,12
3,AE,2022-01-17,5
4,AE,2022-01-24,6


## Join Tables (ActiveSubs & NewSubs)

In [13]:
# Merge active_resampled and new_subs_resampled DataFrames using the Week and country columns as keys
data_join=pd.merge(active_resampled,new_subs_resampled, how="left", on=["Week","country"])

# Replace NaN values with 0
data_join.fillna(0,inplace=True)

# Print the shape of the merged DataFrame
print(f"Shape of data_join: {data_join.shape}")

# Print the first 5 rows of the merged DataFrame
data_join.head()


Shape of data_join: (5703, 4)


,country,Week,ActiveSubs,NewSubs
0,AE,2021-12-27,176.0,2.0
1,AE,2022-01-03,175.0,6.0
2,AE,2022-01-10,173.0,12.0
3,AE,2022-01-17,166.0,5.0
4,AE,2022-01-24,164.0,6.0


## GlobalComparisionIndex

'GlobalComparisionIndex'
It is calculated as "weekly feature value for a country / weekly feature value for global".
On the other hand, we would like to use the metric for solely evaluating a selected country’s performance. 
Therefore; we need to exclude the selected country’s impact on the global sum while calculating the metric for each country.

In [14]:
# Create a copy of the data_join DataFrame
data_for_GCI = data_join.copy(deep=True)

# Group the data_for_GCI DataFrame by Week and compute the sum of each column
data_for_GCI_sum = data_for_GCI.groupby("Week").sum()

# Print the shape of the data_for_GCI_sum DataFrame and show the first five rows
print(f"Shape of data_for_GCI_sum: {data_for_GCI_sum.shape}")
data_for_GCI_sum.head()


Shape of data_for_GCI_sum: (49, 2)


C:\Users\bal_ozgur\AppData\Local\Temp\ipykernel_4840\1587153199.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_for_GCI_sum = data_for_GCI.groupby("Week").sum()


,ActiveSubs,NewSubs
Week,,
2021-12-27,20652.0,152.0
2022-01-03,20297.0,802.0
2022-01-10,20056.0,826.0
2022-01-17,19810.0,792.0
2022-01-24,19751.0,837.0


In [15]:
# Merge data_for_GCI with the summed data_for_GCI_sum on Week column, and fill missing values with 0
data_for_GCI=pd.merge(data_for_GCI, data_for_GCI_sum, how="left", on=["Week"]).fillna(0)

# Print the shape of data_for_GCI DataFrame
print(f"Shape of data_for_GCI :{data_for_GCI.shape}")

# Print the first five rows of data_for_GCI DataFrame
data_for_GCI.head()


Shape of data_for_GCI :(5703, 6)


,country,Week,ActiveSubs_x,NewSubs_x,ActiveSubs_y,NewSubs_y
0,AE,2021-12-27,176.0,2.0,20652.0,152.0
1,AE,2022-01-03,175.0,6.0,20297.0,802.0
2,AE,2022-01-10,173.0,12.0,20056.0,826.0
3,AE,2022-01-17,166.0,5.0,19810.0,792.0
4,AE,2022-01-24,164.0,6.0,19751.0,837.0


In [16]:
# Calculate GCI for new subscribers
data_for_GCI["GCI_NewSubs"] = data_for_GCI["NewSubs_x"] / (data_for_GCI["NewSubs_y"] - data_for_GCI["NewSubs_x"])

# Calculate GCI for active subscribers
data_for_GCI["GCI_ActiveSubs"] = data_for_GCI["ActiveSubs_x"] / (data_for_GCI["ActiveSubs_y"] - data_for_GCI["ActiveSubs_x"])

# Fill NaN values with 0
data_for_GCI.fillna(0, inplace=True)

# Drop unnecessary columns
data_for_GCI.drop(["ActiveSubs_x", "ActiveSubs_y", "NewSubs_x", "NewSubs_y"], axis=1, inplace=True)

# Print the shape of the resulting DataFrame
print(f"Shape of data_for_GCI :{data_for_GCI.shape}")

# Display the first few rows of the resulting DataFrame
data_for_GCI.head()


Shape of data_for_GCI :(5703, 4)


,country,Week,GCI_NewSubs,GCI_ActiveSubs
0,AE,2021-12-27,0.013333,0.008595
1,AE,2022-01-03,0.007538,0.008697
2,AE,2022-01-10,0.014742,0.008701
3,AE,2022-01-17,0.006353,0.008450
4,AE,2022-01-24,0.007220,0.008373


In [17]:
# Merge the data_join and data_for_GCI DataFrames using a left join on "Week" and "country" columns
data_except_z = pd.merge(data_join, data_for_GCI, how="left", on=["Week","country"])

# Print the shape of the merged DataFrame
print(f"Shape of data_except_z :{data_except_z.shape}")

# Display the first 10 rows of the merged DataFrame
data_except_z.head(10)


Shape of data_except_z :(5703, 6)


,country,Week,ActiveSubs,NewSubs,GCI_NewSubs,GCI_ActiveSubs
0,AE,2021-12-27,176.0,2.0,0.013333,0.008595
1,AE,2022-01-03,175.0,6.0,0.007538,0.008697
2,AE,2022-01-10,173.0,12.0,0.014742,0.008701
3,AE,2022-01-17,166.0,5.0,0.006353,0.008450
4,AE,2022-01-24,164.0,6.0,0.007220,0.008373
5,AE,2022-01-31,171.0,8.0,0.008574,0.008718
6,AE,2022-02-07,170.0,6.0,0.007802,0.008736
7,AE,2022-02-14,165.0,4.0,0.005525,0.008556
8,AE,2022-02-21,162.0,5.0,0.006305,0.008444
9,AE,2022-02-28,161.0,6.0,0.007389,0.008439


## Z Score


In [18]:
def z_score(data, window):         
    """Calculate the z-score of a series or array-like object using a rolling window of a specified size.

    Parameters:
    data (pandas.Series or array-like): The data to be transformed.
    window (int): The size of the rolling window.

    Returns:
    pandas.Series: The z-scored data.

    """
    x = data.rolling(window=window) # apply a rolling window to the data
    x_mean = x.mean() # calculate the rolling mean
    x_std = x.std() # calculate the rolling standard deviation
    z = (data-x_mean)/x_std # calculate the z-score
    return z


In [19]:
data_z=data_except_z.copy(deep=True)

In [20]:
# Calculate the z-score for the "GCI_NewSubs" column with a rolling window of 5
data_z["NewSubsGCIZscore"]=z_score(data=data_z["GCI_NewSubs"],window=5)  

# Calculate the z-score for the "GCI_ActiveSubs" column with a rolling window of 5
data_z["ActiveSubsGCIZscore"]=z_score(data=data_z["GCI_ActiveSubs"],window=5)

# Print the first 15 rows of the resulting DataFrame
data_z.head(15)


,country,Week,ActiveSubs,NewSubs,GCI_NewSubs,GCI_ActiveSubs,NewSubsGCIZscore,ActiveSubsGCIZscore
0,AE,2021-12-27,176.0,2.0,0.013333,0.008595,NaN,NaN
1,AE,2022-01-03,175.0,6.0,0.007538,0.008697,NaN,NaN
2,AE,2022-01-10,173.0,12.0,0.014742,0.008701,NaN,NaN
3,AE,2022-01-17,166.0,5.0,0.006353,0.008450,NaN,NaN
4,AE,2022-01-24,164.0,6.0,0.007220,0.008373,-0.672627,-1.292479
5,AE,2022-01-31,171.0,8.0,0.008574,0.008718,-0.092327,0.797962
6,AE,2022-02-07,170.0,6.0,0.007802,0.008736,-0.339708,0.823227
7,AE,2022-02-14,165.0,4.0,0.005525,0.008556,-1.312990,-0.067832
8,AE,2022-02-21,162.0,5.0,0.006305,0.008444,-0.648316,-0.751256
9,AE,2022-02-28,161.0,6.0,0.007389,0.008439,0.222999,-0.971062


In [21]:
# Save the data to a CSV file named "case_1_output.csv"
# Fill NaN values with 0 before saving
# data_z.fillna(0).to_csv("case_1_output.csv", index=False)